In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

In [2]:
import env

In [3]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [17]:
sql = '''
select *
from logs
join cohorts on logs.cohort_id = cohorts.id;
'''

In [18]:
df = pd.read_sql(sql, get_connection('curriculum_logs'))

In [19]:
df.head(10)

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2
5,2018-01-26,09:56:41,javascript-i/loops,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2
6,2018-01-26,09:56:46,javascript-i/conditionals,3,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2
7,2018-01-26,09:56:48,javascript-i/functions,3,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2
8,2018-01-26,09:56:59,javascript-i/loops,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2
9,2018-01-26,09:58:26,javascript-i/functions,4,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2


In [9]:
df.shape

(900223, 4)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   page_viewed  900222 non-null  object 
 1   user_id      900223 non-null  int64  
 2   cohort_id    847330 non-null  float64
 3   ip           900223 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 34.3+ MB


In [11]:
df.describe()

,user_id,cohort_id
count,900223.000000,847330.000000
mean,458.825707,48.501049
std,249.296767,32.795482
min,1.000000,1.000000
25%,269.000000,28.000000
50%,475.000000,33.000000
75%,660.000000,57.000000
max,981.000000,139.000000


In [12]:
df.isnull().sum()

page_viewed        1
user_id            0
cohort_id      52893
ip                 0
dtype: int64

In [13]:
df['cohort_id'] = df.cohort_id.fillna(0)


In [14]:
df.isnull().sum()

page_viewed    1
user_id        0
cohort_id      0
ip             0
dtype: int64

In [15]:
for col in df.columns:
    print(col)
    print(df[col].value_counts(),'\n')

page_viewed
/                                        50313
search/search_index.json                 19519
javascript-i                             18983
toc                                      18297
java-iii                                 13733
                                         ...  
11-nlp/3-acquire/.json                       1
11-distributed-ml/6.2-prepare-part-2         1
4-stats/2.5-distributions-and-testing        1
python                                       1
jquery/whack-a-mole                          1
Name: page_viewed, Length: 2313, dtype: int64 

user_id
11     17913
64     16347
53     12329
314     7783
1       7404
       ...  
952        1
212        1
940        1
649        1
879        1
Name: user_id, Length: 981, dtype: int64 

cohort_id
28.0     84031
0.0      52893
33.0     40730
29.0     38096
62.0     37109
53.0     36902
24.0     35636
57.0     33844
56.0     33568
51.0     32888
59.0     32015
22.0     30926
58.0     29855
32.0     29356
23.0     